<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/Multicompartment_Sim_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Multicompartment Simulator**

**Eran Frank Shorer**

**MSc Computational Neuroscience**

**"Investigating the effects of impermeant anions on the electrical and computational properties of neurons"**




## Instructions:

This jupyter notebook consists of cells. Some cells are plain text, while others are code. 

Press Shift-Enter simultaneously, or click the cell and press the play button in the main toolbar above to run the cell. 

The output of the cell should be displayed below the cell. 



## **Import classes and modules** 


In [8]:
from ipywidgets import widgets, Layout, interact, interactive, interactive_output
from IPython.display import display
import common
import constants
import compartment
import electrodiffusion
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
print("All relevant classes imported")


All relevant classes imported


## **Set simulation parameters**

## *A) Timing*

In [55]:
lbl_total_t = widgets.Label(value="Set the simulation run time (minutes):")
sldr_total_t = widgets.FloatSlider(value=3, min=1, max=20, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
lbl_dt = widgets.Label(value="Set the simulation time step (milliseconds):")
sldr_dt = widgets.FloatSlider(value=1, min=1, max=10, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')

dt= sldr_dt.value*1e-3 # 1ms time steps 
total_t= sldr_total_t.value *60 #s

run_t = 0 #current simulation timing
t_arr = [0]


widgets.VBox([lbl_total_t,sldr_total_t,lbl_dt,sldr_dt], layout=Layout(border='solid'))


#print("Set total simulation time (minutes):")

#print("Set time step (milliseconds):")


## *B) Compartment parameters*

### Basic compartment parameters:

In [58]:
txt_name = widgets.Text(value="Comp1")
lbl_name = widgets.Label(value ="Compartment Name: ")
name_box = widgets.HBox(children = [lbl_name,txt_name])

lbl_rad = widgets.Label(value = "Compartment Radius (um): ")
sldr_rad = widgets.FloatSlider(value=5, min=1, max=11.0, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
rad_box = widgets.HBox(children = [lbl_rad,sldr_rad])

lbl_len = widgets.Label(value = "Compartment Length (um): ")
sldr_len = widgets.FloatSlider(value=100, min=50, max=200, step=20, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
len_box = widgets.HBox(children = [lbl_len,sldr_len])

global name, comp_arr
name = txt_name.value
comp_arr =[]

output = widgets.Output()


btn_new_comp = widgets.Button(description = "Create Compartment",button_style ="success")


def new_comp_clicked(b):
    
    name = txt_name.value
    new_comp = compartment.Compartment(name,radius=sldr_rad.value*1e-5, length=sldr_len.value*1e-5)
    comp_arr.append(new_comp)
    with output:
        print("Compartment: ("+ new_comp.name +") created" )

      
btn_new_comp.on_click(new_comp_clicked)


vbox_grand = widgets.VBox(children = [name_box,rad_box,len_box,btn_new_comp], layout=Layout(border ="solid"))

display(output,vbox_grand)







Output()

In [49]:
x = 0
for i in comp_arr:
    print(comp_arr[x].name + ": " + str(comp_arr[x].length)+ " um")
    x += 1

Comp1: 0.0013000000000000002 um
Comp1: 0.0017000000000000001 um


In [19]:
comp_1 = compartment.Compartment("comp_1")
comp_1.set_ion_properties()
comp_2 = compartment.Compartment("comp_2")
comp_2.set_ion_properties()
ed_1_2 = electrodiffusion.Electrodiffusion(comp_1,comp_2)

## **Run Simulation**

In [13]:
while run_t < total_t:

    comp_1.step(dt)
    comp_2.step(dt)
    comp_1_ed_dict = comp_1.get_ed_dict()
    comp_2_ed_dict = comp_2.get_ed_dict()
    ed_conc_changes = ed_1_2.calc_ed(dt,comp_1_ed_dict,comp_2_ed_dict)
    comp_1.ed_update(ed_conc_changes)
    for j in ed_conc_changes:
        ed_conc_changes[j] *= -1
    comp_2.ed_update(ed_conc_changes)
    comp_1.update_volumes()
    comp_1.update_arrays()
    run_t += dt
    t_arr.append(run_t)



TypeError: 'dict_keys' object is not subscriptable